In [1]:
import cirq
import numpy as np
import sympy
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
import time
from scipy.optimize import minimize

In [2]:
# Configuration
n_qubits = 2
n_layers = 2
noise_type = "amplitude_damping"   # options: "bit_flip","phase_flip","amplitude_damping","phase_damping","depolarizing", or None
noise_prob = 0.05
epochs = 30
lr = 0.05

# Load binary iris (first two classes)
iris = load_iris()
mask = iris.target < 2
X = iris.data[mask]
y = iris.target[mask].astype(float)  # {0,1}

# Use first n_qubits features if available, otherwise tile features to match n_qubits
X_raw = X
if X_raw.shape[1] >= n_qubits:
    X = X_raw[:, :n_qubits]
else:
    reps = int(np.ceil(n_qubits / X_raw.shape[1]))
    X = np.tile(X_raw, reps)[:, :n_qubits]

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Create qubits
qubits = [cirq.GridQubit(0, i) for i in range(n_qubits)]

def add_noise_channel(circuit, prob, qubits):
    """Add noise channels to the circuit"""
    if noise_type is None or noise_type.lower() == "none":
        return
    
    for qubit in qubits:
        t = noise_type.lower()
        if t == "bit_flip":
            circuit.append(cirq.bit_flip(prob)(qubit))
        elif t == "phase_flip":
            circuit.append(cirq.phase_flip(prob)(qubit))
        elif t == "amplitude_damping":
            circuit.append(cirq.amplitude_damp(prob)(qubit))
        elif t == "phase_damping":
            circuit.append(cirq.phase_damp(prob)(qubit))
        elif t == "depolarizing":
            circuit.append(cirq.depolarize(prob)(qubit))
        else:
            raise ValueError(f"Unknown noise type: {noise_type}")

def angle_embedding(circuit, inputs, qubits):
    """Embed input data as rotation angles"""
    for i, qubit in enumerate(qubits):
        if i < len(inputs):
            circuit.append(cirq.ry(inputs[i])(qubit))

def strongly_entangling_layer(circuit, weights, qubits, layer_idx):
    """Implement strongly entangling layer similar to PennyLane"""
    n_qubits = len(qubits)
    
    # Apply rotations to each qubit
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(weights[layer_idx, i, 0])(qubit))
        circuit.append(cirq.ry(weights[layer_idx, i, 1])(qubit))
        circuit.append(cirq.rz(weights[layer_idx, i, 2])(qubit))
    
    # Add entangling gates (CNOT between adjacent qubits + wrap-around)
    for i in range(n_qubits):
        control = qubits[i]
        target = qubits[(i + 1) % n_qubits]
        circuit.append(cirq.CNOT(control, target))

def create_circuit(inputs, weights, qubits, add_noise=True):
    """Create the full quantum circuit"""
    circuit = cirq.Circuit()
    
    # Angle embedding
    angle_embedding(circuit, inputs, qubits)
    
    # Strongly entangling layers
    for layer in range(n_layers):
        strongly_entangling_layer(circuit, weights, qubits, layer)
    
    # Add noise after ansatz
    if add_noise:
        add_noise_channel(circuit, noise_prob, qubits)
    
    # Measurement
    circuit.append(cirq.measure(qubits[0], key='z'))
    
    return circuit

def get_expectation_z(inputs, weights, qubits, shots=1000):
    """Get expectation value of Pauli-Z on first qubit"""
    circuit = create_circuit(inputs, weights, qubits, add_noise=True)
    
    # For noisy simulation, we need to use density matrix simulator
    if noise_type is not None and noise_type.lower() != "none":
        simulator = cirq.DensityMatrixSimulator()
        # Remove measurement for expectation calculation
        circuit_no_measure = circuit[:-1]  # Remove the measurement
        result = simulator.simulate(circuit_no_measure)
        # Calculate expectation value of Z on first qubit
        z_op = cirq.Z(qubits[0])
        expectation = z_op.expectation_from_density_matrix(
            result.final_density_matrix, 
            qubit_map={q: i for i, q in enumerate(qubits)}
        ).real
    else:
        # For noiseless case, can use sampling
        simulator = cirq.Simulator()
        results = simulator.run(circuit, repetitions=shots)
        measurements = results.measurements['z']
        # Convert 0,1 measurements to -1,+1 for expectation
        z_values = 2 * measurements.flatten() - 1
        expectation = np.mean(z_values)
    
    return expectation

def cost_function(weights_flat, X_batch, y_batch):
    """Cost function for optimization"""
    weights = weights_flat.reshape(n_layers, n_qubits, 3)
    preds = []
    
    for x in X_batch:
        # Pad/tile input to match n_qubits
        if len(x) < n_qubits:
            reps = int(np.ceil(n_qubits / len(x)))
            x_padded = np.tile(x, reps)[:n_qubits]
        else:
            x_padded = x[:n_qubits]
        
        expectation = get_expectation_z(x_padded, weights, qubits)
        pred = (expectation + 1.0) / 2.0  # Map from [-1,1] to [0,1]
        preds.append(pred)
    
    preds = np.array(preds)
    mse = np.mean((preds - y_batch) ** 2)
    return mse

# Initialize weights
weights_init = np.random.uniform(0, 2 * np.pi, size=(n_layers, n_qubits, 3))
weights_flat = weights_init.flatten()

In [4]:
print("Starting training...")

# Training with gradient-free optimization (since gradients are complex with noise)
def callback(xk):
    """Callback function to print progress"""
    global iteration_count
    if iteration_count % 5 == 0:
        loss = cost_function(xk, X_train, y_train)
        print(f"Iteration {iteration_count:3d} | Train Loss: {loss:.6f}")
    iteration_count += 1

iteration_count = 0

# Use COBYLA optimizer which works well for noisy quantum circuits
result = minimize(
    fun=cost_function,
    x0=weights_flat,
    args=(X_train, y_train),
    method='COBYLA',
    options={'maxiter': epochs, 'disp': True},
    callback=callback
)

optimal_weights = result.x.reshape(n_layers, n_qubits, 3)

print("\nTraining completed!")

Starting training...
Iteration   0 | Train Loss: 0.391742
Iteration   5 | Train Loss: 0.137232
Iteration  10 | Train Loss: 0.131449
Iteration  15 | Train Loss: 0.130506
Iteration  20 | Train Loss: 0.124342
Iteration  25 | Train Loss: 0.142238

   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =   30   F = 1.214893E-01    MAXCV = 0.000000E+00
   X = 2.047001E+00   1.456801E+00   2.731877E+00   2.378747E-01   6.617732E+00
       5.873917E+00   4.446423E+00   2.200082E+00   1.314789E+00   2.750984E+00
       6.238678E+00   5.884244E+00
Iteration  30 | Train Loss: 0.121489

Training completed!


In [5]:
# Evaluation
print("\nEvaluating on test set...")
t0 = time.perf_counter()

preds_test = []
for x in X_test:
    # Pad/tile input to match n_qubits
    if len(x) < n_qubits:
        reps = int(np.ceil(n_qubits / len(x)))
        x_padded = np.tile(x, reps)[:n_qubits]
    else:
        x_padded = x[:n_qubits]
    
    expectation = get_expectation_z(x_padded, optimal_weights, qubits)
    pred = (expectation + 1.0) / 2.0
    preds_test.append(pred)

t1 = time.perf_counter()
preds_test = np.array(preds_test)

# Threshold for binary classification
preds_binary = (preds_test > 0.5).astype(int)
y_true_int = np.array(y_test).astype(int)

# Metrics
mse = mean_squared_error(y_true_int, preds_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_int, preds_test)
f1 = f1_score(y_true_int, preds_binary)
inference_time_per_sample = (t1 - t0) / len(X_test)

print("\n=== SELA-P Cirq Evaluation ===")
print(f"Noise type: {noise_type}")
print(f"Noise prob: {noise_prob}")
print(f"# qubits: {n_qubits}")
print(f"MSE:  {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAE:  {mae:.6f}")
print(f"F1-score:  {f1:.6f}")
print(f"Inference time per sample: {inference_time_per_sample*1000:.4f} ms")


Evaluating on test set...

=== SELA-P Cirq Evaluation ===
Noise type: amplitude_damping
Noise prob: 0.05
# qubits: 2
MSE:  0.148971
RMSE: 0.385967
MAE:  0.348747
F1-score:  0.800000
Inference time per sample: 2.4858 ms
